In [ ]:
%pip install pandas morfessor 

: 

In [ ]:
import pandas as pd
import morfessor


input_tsv = 'data\eng.sentence.test.gold.tsv'  #path to the input tsv file
training_txt = 'data\\training_morfessor.txt' #path to the txt file
output_tsv = 'outputs\eng.sentence.test.morfessor_guess.tsv'  #path tot the output tsv file


io = morfessor.MorfessorIO()
model = morfessor.BaselineModel()


df = pd.read_csv(input_tsv, sep='\t', header=None)
data = df[0].astype(str)  #data frame with the sentences

with open(training_txt, 'w', encoding='utf-8') as f:  #write training data into txt file
  for entry in data:
    words = entry.split()
    for word in words:
      f.write(word + '\n')



training_data = io.read_corpus_file(training_txt)
model.load_data(training_data)
model.train_batch


df_guess = df
i = 0
for entry in data:
  sent = ''
  list_sent = model.viterbi_segment(entry)[0] #list of segmented sentence
  j = 0
  length = len(list_sent)
  for word in list_sent:
    if j != length - 1:
      sent += (word + ' @@')
      j+=1
    else :
      sent += word
  df_guess[1][i] = sent
  i+=1

df_guess.to_csv(output_tsv, sep='\t', header=None, index = False)